In [ ]:
import pandas as pd
import json
import time
import logging
import os
from underthesea import word_tokenize
import re
import unicodedata

In [14]:
# Cấu hình logging
logging.basicConfig(
    filename='processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w',  # Ghi đè file log mỗi lần chạy
    encoding='utf-8'
)

In [15]:
# Danh sách từ viết tắt/lóng phổ biến trên mạng xã hội
COMMON_ABBREVIATIONS = {
    'ad', 'ib', 'tt', 'rep', 'dm', 'pm', 'acc', 'fb', 'zalo', 'ok', 'ko', 'dc',
    'ns', 'vs', 'nt', 'cm', 'sp', 'pr', 'hot', 'top', 'vcl', 'wtf', 'lol', 'cfs'
}

In [16]:
def detect_abbreviations(post_text, index):
    """
    Phát hiện các từ viết tắt hoặc thuật ngữ/ký hiệu không rõ ràng trong nội dung bài viết.
    Xử lý Post Text là danh sách chuỗi.
    Sử dụng underthesea để phân đoạn từ và danh sách từ viết tắt phổ biến.
    Trả về danh sách các từ viết tắt duy nhất hoặc danh sách rỗng nếu không tìm thấy.
    Ghi log lỗi nếu xảy ra.
    """
    try:
        # Nếu post_text là danh sách, nối các chuỗi thành một chuỗi
        if isinstance(post_text, list):
            post_text = " ".join(str(text) for text in post_text if pd.notnull(text))
        else:
            post_text = str(post_text) if pd.notnull(post_text) else ""

        # Phân đoạn từ bằng underthesea
        tokens = word_tokenize(post_text, format="text").split()
        # Lọc các từ ngắn (dưới 4 ký tự) hoặc trong danh sách từ viết tắt
        abbreviations = set()
        for token in tokens:
            token = token.lower().strip()
            # Kiểm tra từ ngắn hoặc trong danh sách từ viết tắt
            if (len(token) <= 4 or token in COMMON_ABBREVIATIONS) and token.isalpha():
                abbreviations.add(token)
        return list(abbreviations)
    except Exception as e:
        logging.error(f"Lỗi khi xử lý văn bản tại index {index}: {str(e)}")
        print(f"Lỗi tại index {index}: {str(e)}")
        return []

In [17]:
def detect_file_encoding(file_path):
    """
    Phát hiện mã hóa của file bằng charset-normalizer.
    Trả về mã hóa được phát hiện hoặc None nếu thất bại.
    """
    try:
        with open(file_path, 'rb') as f:
            raw_data = f.read()
        result = detect(raw_data)
        encoding = result.get('encoding')
        logging.info(f"Phát hiện mã hóa của file {file_path}: {encoding}")
        return encoding
    except Exception as e:
        logging.error(f"Lỗi khi phát hiện mã hóa của file {file_path}: {str(e)}")
        print(f"Lỗi khi phát hiện mã hóa của file {file_path}: {str(e)}")
        return None

In [18]:
def process_json_file(input_file, output_file):
    """
    Xử lý file JSON để phát hiện các từ viết tắt và lưu kết quả vào file JSON.
    Ghi log tiến độ, lỗi, thời gian thực hiện và ETA theo thời gian thực.
    Hỗ trợ thử nhiều mã hóa khi đọc file JSON.
    """
    # Đọc file JSON
    encodings_to_try = ['utf-8', 'utf-16', 'latin1', 'cp1252']
    data = None
    for encoding in encodings_to_try:
        try:
            with open(input_file, 'r', encoding=encoding) as f:
                data = json.load(f)
            logging.info(f"Đọc file JSON thành công với mã hóa {encoding}")
            break
        except UnicodeDecodeError as e:
            logging.warning(f"Thất bại khi đọc file JSON với mã hóa {encoding}: {str(e)}")
            continue
        except Exception as e:
            logging.error(f"Lỗi khi đọc file JSON với mã hóa {encoding}: {str(e)}")
            continue

    # Nếu không đọc được với bất kỳ mã hóa nào
    if data is None:
        # Thử phát hiện mã hóa bằng charset-normalizer
        detected_encoding = detect_file_encoding(input_file)
        if detected_encoding:
            try:
                with open(input_file, 'r', encoding=detected_encoding) as f:
                    data = json.load(f)
                logging.info(f"Đọc file JSON thành công với mã hóa được phát hiện: {detected_encoding}")
            except Exception as e:
                error_msg = f"Lỗi khi đọc file JSON với mã hóa được phát hiện {detected_encoding}: {str(e)}"
                logging.error(error_msg)
                print(error_msg)
                return
        else:
            error_msg = f"Không thể đọc file JSON {input_file} với bất kỳ mã hóa nào"
            logging.error(error_msg)
            print(error_msg)
            return

    try:
        df = pd.DataFrame(data)
    except Exception as e:
        logging.error(f"Lỗi khi chuyển JSON thành DataFrame: {str(e)}")
        print(f"Lỗi khi chuyển JSON thành DataFrame: {str(e)}")
        return

    # Xác minh các cột bắt buộc
    required_columns = ['index', 'Page URL', 'Page Name', 'Post URL', 'Post Text']
    if not all(col in df.columns for col in required_columns):
        error_msg = f"File JSON thiếu một hoặc nhiều cột bắt buộc: {required_columns}"
        logging.error(error_msg)
        print(error_msg)
        return

    # Danh sách lưu kết quả
    results = []
    total_rows = len(df)  # Tổng số dòng trong file JSON
    processed_rows = 0    # Số dòng đã xử lý
    row_times = []        # Lưu thời gian xử lý mỗi dòng để tính ETA
    start_time = time.time()

    logging.info(f"Bắt đầu xử lý {total_rows} dòng dữ liệu")

    # Xử lý từng dòng
    for _, row in df.iterrows():
        row_start_time = time.time()
        index = int(row['index'])  # Chuyển index từ chuỗi sang số nguyên

        # Lấy nội dung Post Text
        post_text = row['Post Text']
        abbreviations = detect_abbreviations(post_text, index) if post_text else []

        # Tạo bản ghi kết quả
        result = {
            "index": index,
            "Page URL": str(row['Page URL']),
            "Page Name": str(row['Page Name']),
            "Post URL": str(row['Post URL']),
            "Post Text": post_text,  # Giữ nguyên dạng danh sách trong đầu ra
            "Abbreviation": abbreviations
        }
        results.append(result)

        # Tăng số dòng đã xử lý
        processed_rows += 1
        # Tính thời gian xử lý dòng hiện tại và cập nhật danh sách
        row_time = time.time() - row_start_time
        row_times.append(row_time)
        # Lấy trung bình động (moving average) của 10 dòng gần nhất để tính ETA
        avg_time_per_row = sum(row_times[-10:]) / min(len(row_times), 10)
        eta = avg_time_per_row * (total_rows - processed_rows)
        elapsed_time = time.time() - start_time
        # Ghi log và in tiến độ
        progress_msg = (f"Đã xử lý {processed_rows}/{total_rows} dòng (Index: {index}) "
                       f"| Thời gian đã trôi qua: {elapsed_time:.2f}s | ETA: {eta:.2f}s")
        logging.info(progress_msg)
        print(progress_msg)

    # Ghi kết quả ra file JSON
    try:
        os.makedirs(os.path.dirname(output_file), exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        logging.info(f"Kết quả đã được lưu vào {output_file}")
        print(f"Kết quả đã được lưu vào {output_file}")
    except Exception as e:
        logging.error(f"Lỗi khi ghi file JSON: {str(e)}")
        print(f"Lỗi khi ghi file JSON: {str(e)}")


In [19]:
if __name__ == "__main__":
    input_file = "../../Facebook Page Posts Scraping/Clean Data/output/Confessions of HNMU.xlsx"
    output_file = "output/Confessions of HNMU.json"
    process_json_file(input_file, output_file)

Lỗi khi phát hiện mã hóa của file ../../Facebook Page Posts Scraping/Clean Data/output/Confessions of HNMU.xlsx: name 'detect' is not defined
Không thể đọc file JSON ../../Facebook Page Posts Scraping/Clean Data/output/Confessions of HNMU.xlsx với bất kỳ mã hóa nào
